# Food Safety Ratings Norwich: ETL

Fetch data using Food standards agency API. 
## API info
---------
**Version: 2**

Endpoints used:
- [ScoreDescriptors](http://api.ratings.food.gov.uk/ScoreDescriptors): Table with descriptions of confidence in management, structural and hygiene scores. 
- [Establishments](http://api.ratings.food.gov.uk/Establishments?localAuthorityId=33&pageNumber=0): Table with details of Establishments and associated data.

------------


In [1]:
import pandas as pd
import numpy as np
import requests
import re

%load_ext watermark
pd.set_option("display.max_columns", 40)

In [2]:
%watermark -iv -dt

requests: 2.25.1
re      : 2.2.1
numpy   : 1.20.1
pandas  : 1.4.3
sys     : 3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]



In [3]:
class FoodHygieneRatingAPI:
    def __init__(self):
        self.base_url = "http://api.ratings.food.gov.uk/"

    def get_scores_descriptions(self, as_frame=True):
        """get description of scores

        Explanation of `hygiene`, `structural` and `confidence in management` scores
        """
        with requests.Session() as s:
            scores_desc = s.get(
                self.base_url + "ScoreDescriptors",
                headers={"accept": "application/json", "x-api-version": "2"},
            )
            if scores_desc.reason != "OK":
                scores_desc.raise_for_status()

        if as_frame:
            return (
                pd.json_normalize(scores_desc.json(), "scoreDescriptors")
                .pivot_table(
                    columns="ScoreCategory",
                    index="Description",
                    values="Score",
                    dropna=False,
                    fill_value="-",
                )
                .sort_values(by="Structural")
            )
        else:
            return scores_desc.json()

    def get_establishments_data(self, as_frame=True):
        """get table"""
        with requests.Session() as s:
            establishment_food_rating = s.get(
                self.base_url + "Establishments?localAuthorityId=33&pageNumber=0",
                headers={"x-api-version": "2", "accept": "application/json"},
            )
            if establishment_food_rating.reason != "OK":
                establishment_food_rating.raise_for_status()

        if as_frame:
            return pd.json_normalize(establishment_food_rating.json(), "establishments")
        else:
            return establishment_food_rating.json()

    @staticmethod
    def get_post_codes(location="NR"):
        return pd.read_csv(
            f"https://raw.githubusercontent.com/odileeds/Postcodes2LatLon/master/postcodes/{location}.csv"
        ).rename(
            columns={"Postcode": "post_code"}
        )  

In [4]:
food_api = FoodHygieneRatingAPI()

food_ratings = food_api.get_establishments_data()
food_ratings.head()

,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,SchemeType,RightToReply,Distance,NewRatingPending,links,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,24962,0,08/00030/FD_HS,38 St Giles Boutique Bed And Breakfrast,Hotel/bed & breakfast/guest house,7842,38 St Giles Street,Norwich,,,NR2 1LL,,5,fhrs_5_en-gb,2019-10-09T00:00:00,232,Norwich City,https://www.norwich.gov.uk/site/custom_scripts...,foodandsafety@norwich.gov.uk,FHRS,,None,False,"[{'rel': 'self', 'href': 'http://api.ratings.f...",0.0,0.0,5.0,1.289821,52.629101,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,915695,0,16/00123/FD_HS,9 Bar,Pub/bar/nightclub,7843,37 Prince Of Wales Road,Norwich,,,NR1 1BG,,5,fhrs_5_en-gb,2019-03-13T00:00:00,232,Norwich City,https://www.norwich.gov.uk/site/custom_scripts...,foodandsafety@norwich.gov.uk,FHRS,,None,False,"[{'rel': 'self', 'href': 'http://api.ratings.f...",5.0,5.0,5.0,1.300907,52.629297,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,1410235,0,21/00298/FD_HS,A \& M Events (@ Epic),Restaurant/Cafe/Canteen,1,112 - 114 Magdalen Street,Norwich,,,NR3 1JD,,5,fhrs_5_en-gb,2021-11-30T00:00:00,232,Norwich City,https://www.norwich.gov.uk/site/custom_scripts...,foodandsafety@norwich.gov.uk,FHRS,,None,False,"[{'rel': 'self', 'href': 'http://api.ratings.f...",0.0,0.0,0.0,1.296628,52.638032,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,33349,0,10/00039/FD_HS,Abbeyfield,Caring Premises,5,57 The Close,Norwich,,,NR1 4EH,,5,fhrs_5_en-gb,2018-05-03T00:00:00,232,Norwich City,https://www.norwich.gov.uk/site/custom_scripts...,foodandsafety@norwich.gov.uk,FHRS,,None,False,"[{'rel': 'self', 'href': 'http://api.ratings.f...",5.0,5.0,0.0,1.302585,52.631948,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,579830,0,13/00225/FOOD,Abellio Greater Anglia,Takeaway/sandwich shop,7844,Norwich Railway Station,Station Approach,Norwich,,NR1 1EF,,5,fhrs_5_en-gb,2018-11-19T00:00:00,232,Norwich City,https://www.norwich.gov.uk/site/custom_scripts...,foodandsafety@norwich.gov.uk,FHRS,,None,False,"[{'rel': 'self', 'href': 'http://api.ratings.f...",5.0,5.0,0.0,1.306479,52.626938,None,0001-01-01T00:00:00,0,None,0,0,0,0


In [5]:
food_ratings.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1657 entries, 0 to 1656
Columns: 37 entries, FHRSID to meta.pageNumber
dtypes: bool(1), float64(3), int64(8), object(25)
memory usage: 467.8+ KB


In [6]:
food_api.get_scores_descriptions()

ScoreCategory,Confidence,Hygiene,Structural
Description,,,
Very good,0.0,0.0,0.0
Good,5.0,5.0,5.0
Generally satisfactory,10.0,10.0,10.0
Improvement necessary,-,15.0,15.0
Major improvement necessary,20.0,20.0,20.0
Urgent improvement necessary,30.0,25.0,25.0


In [7]:
(
    pd.crosstab(food_ratings.BusinessType, food_ratings.RatingValue, normalize="index")
    .sort_values(by=["0", "1", "2"], ascending=False)
    .style.format(precision=4)
    .background_gradient(subset=["3", "4", "5"], cmap="PuBuGn")
    .highlight_quantile(
        subset=["AwaitingInspection", "Exempt"], color="#ff7f7f", q_left=0.75
    )
)

RatingValue,0,1,2,3,4,5,AwaitingInspection,Exempt
BusinessType,,,,,,,,
Restaurant/Cafe/Canteen,0.0024,0.0049,0.0146,0.0998,0.2555,0.5669,0.0560,0.0000
Takeaway/sandwich shop,0.0000,0.0297,0.0297,0.1832,0.3515,0.3366,0.0644,0.0050
Retailers - other,0.0000,0.0178,0.0030,0.0888,0.2485,0.4083,0.0680,0.1657
Hotel/bed & breakfast/guest house,0.0000,0.0000,0.0333,0.0667,0.1000,0.7333,0.0667,0.0000
Manufacturers/packers,0.0000,0.0000,0.0294,0.0294,0.1471,0.6471,0.0882,0.0588
Caring Premises,0.0000,0.0000,0.0118,0.0588,0.2000,0.6941,0.0235,0.0118
Pub/bar/nightclub,0.0000,0.0000,0.0058,0.1170,0.2690,0.5263,0.0819,0.0000
Distributors/Transporters,0.0000,0.0000,0.0000,0.0714,0.2857,0.5000,0.1429,0.0000
Farmers/growers,0.0000,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000


In [8]:
# cleaning funcs
drop_cols = [
    "FHRSID",
    "ChangesByServerID",
    "LocalAuthorityBusinessID",
    "AddressLine1",
    "AddressLine2",
    "AddressLine3",
    "AddressLine4",
    "Phone",
    "RatingKey",
    "LocalAuthorityCode",
    "LocalAuthorityName",
    "LocalAuthorityWebSite",
    "LocalAuthorityEmailAddress",
    "SchemeType",
    "RightToReply",
    "Distance",
    "links",
    "meta.dataSource",
    "meta.extractDate",
    "meta.itemCount",
    "meta.returncode",
    "meta.totalCount",
    "meta.totalPages",
    "meta.pageSize",
    "meta.pageNumber",
]


def drop_columns(df, cols_to_drop):
    return df.drop(labels=cols_to_drop, axis=1)


def rename_cols(df):
    out = list()
    out_df = df.copy()
    for column in df.columns:
        column = re.sub("\.", "_", column)
        match = re.findall("[A-Z]+", column)[1:]
        for i in match:
            column = re.sub(i, "_" + i.lower(), column)
        out.append(column.lower())
    out_df.columns = out
    return out_df


def convert_dtype(df):
    df = df.astype(
        {
            "geocode_longitude": float,
            "geocode_latitude": float,
            "rating_date": np.datetime64,
        }
    )
    return df


def enrich_geolocation(df):
    """get geolocation of address with no lat/lon using post codes"""
    pc_df = df.merge(FoodHygieneRatingAPI.get_post_codes(), on="post_code", how="inner")

    pc_df[["geocode_longitude", "geocode_latitude"]] = np.where(
        np.isnan(pc_df[["geocode_longitude", "geocode_latitude"]]),
        pc_df[["long", "lat"]],
        pc_df[["geocode_longitude", "geocode_latitude"]],
    )
    pc_df.drop(columns=["long", "lat"], inplace=True)
    return pc_df


def create_dummies_for_non_rating(df, rating_col):
    """Create dummy fields for 'Exempt' and 'AwaitingInspection' fields"""
    dummies = pd.get_dummies(
        df[rating_col], columns=["Exempt", "AwaitingInspection"]
    ).drop(columns=list("012345"))
    out_df = pd.concat([df, dummies], axis=1)
    out_df.rename(
        columns={"AwaitingInspection": "awaiting_inspection", "Exempt": "exempt"},
        inplace=True,
    )
    return out_df


def replace_non_rating(df, rating_col):
    """Replace 'Exempt' and 'AwatingInspection' with NaNs"""
    out_df = df.copy()
    mask = [x in ["Exempt", "AwaitingInspection"] for x in df[rating_col]]
    out_df[rating_col] = np.where(mask, np.nan, df[rating_col]).astype(np.float16)
    return out_df


def drop_invalid_dates(df, date_col):
    out_df = df.copy()
    out_df[date_col] = pd.to_datetime(
        np.where(
            df[date_col].dt.year < 2000,
            pd.to_datetime(np.nan, errors="coerce"),
            df[date_col],
        )
    )
    return out_df


def convert_to_categorical(df):
    object_cols = df.select_dtypes(include="object").head(1).columns
    for col in object_cols:
        df[col] = df[col].astype("category")
    return df

In [9]:
# cleaning pipeline
food_ratings_cleaned = (
    food_ratings.pipe(drop_columns, drop_cols)
    .assign(post_code_outer=food_ratings["PostCode"].str.extract(r"(.*(?<=\s))"))
    .assign(
        business_type_primary=food_ratings["BusinessType"].apply(
            lambda x: re.split(" - |/", x)[0]
        )
    )
    .pipe(rename_cols)
    .pipe(convert_dtype)
    .pipe(enrich_geolocation)
    .pipe(create_dummies_for_non_rating, "rating_value")
    .pipe(replace_non_rating, "rating_value")
    .pipe(drop_invalid_dates, "rating_date")
    .pipe(convert_to_categorical)
)

In [10]:
food_ratings_cleaned.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1474
Columns: 16 entries, business_name to exempt
dtypes: bool(1), category(5), datetime64[ns](1), float16(1), float64(5), int64(1), uint8(2)
memory usage: 175.7 KB


In [11]:
food_ratings_cleaned = food_ratings_cleaned.loc[:, sorted(food_ratings_cleaned.columns)]

In [12]:
food_ratings_cleaned.head(15)

,awaiting_inspection,business_name,business_type,business_type_id,business_type_primary,exempt,geocode_latitude,geocode_longitude,new_rating_pending,post_code,post_code_outer,rating_date,rating_value,scores_confidence_in_management,scores_hygiene,scores_structural
0,0,38 St Giles Boutique Bed And Breakfrast,Hotel/bed & breakfast/guest house,7842,Hotel,0,52.629101,1.289821,False,NR2 1LL,NR2,2019-10-09,5.0,5.0,0.0,0.0
1,0,Catherines Coffee Lounge,Restaurant/Cafe/Canteen,1,Restaurant,0,52.629102,1.289822,False,NR2 1LL,NR2,2022-03-08,5.0,5.0,5.0,5.0
2,0,Chestnut Nursery School,Caring Premises,5,Caring Premises,0,52.629101,1.289821,False,NR2 1LL,NR2,2022-06-27,5.0,5.0,0.0,0.0
3,0,9 Bar,Pub/bar/nightclub,7843,Pub,0,52.629297,1.300907,False,NR1 1BG,NR1,2019-03-13,5.0,5.0,5.0,5.0
4,0,Sugar And Spice,Pub/bar/nightclub,7843,Pub,0,52.629297,1.300907,False,NR1 1BG,NR1,2018-04-16,5.0,5.0,5.0,0.0
5,0,TheFatFoxPizzaDen,Takeaway/sandwich shop,7844,Takeaway,0,52.629296,1.300907,False,NR1 1BG,NR1,2022-06-10,5.0,NaN,NaN,NaN
6,0,Gringos,Pub/bar/nightclub,7843,Pub,0,52.629297,1.300907,False,NR1 1BG,NR1,2018-03-21,4.0,NaN,NaN,NaN
7,0,Tastebuds,Takeaway/sandwich shop,7844,Takeaway,0,52.629296,1.300907,False,NR1 1BG,NR1,2019-08-30,4.0,10.0,5.0,5.0
8,0,Tasty Restaurant,Restaurant/Cafe/Canteen,1,Restaurant,0,52.629297,1.300907,False,NR1 1BG,NR1,2022-05-17,3.0,NaN,NaN,NaN
9,1,Sherbet Lemon,Pub/bar/nightclub,7843,Pub,0,52.629297,1.300907,False,NR1 1BG,NR1,NaT,NaN,NaN,NaN,NaN


In [13]:
food_ratings_cleaned.to_pickle("processed_data/food_ratings.pkl")